In [1]:
#%run ./methods.ipynb

In [3]:
#df = pd.read_pickle('./data/fitbitflexwirelessactivitysleepwristbandblack.pkl')

In [4]:
#%run ./tfidf.ipynb

In [5]:
entities = []
entities_scores = []
phrases = []
for index, review in df.iterrows():
    tmp_entities = []
    tmp_entities_scores = []
    tmp_phrases = []
    all_entities = review.noun_phrase_entities+review.bigram_entities+review.trigram_entities
    all_phrases = review.noun_phrases+review.bigrams+review.trigrams
    all_scores = review.noun_phrase_scores+review.bigrams_scores+review.trigrams_scores
    for i, entity in enumerate(all_entities):
        if entity != '':
            if len(nltk.word_tokenize(all_phrases[i])) >= 2:
                tmp_entities.append(entity)
                tmp_entities_scores.append(all_scores[i])
                tmp_phrases.append(all_phrases[i])
    entities.append(tmp_entities)
    entities_scores.append(tmp_entities_scores)
    phrases.append(tmp_phrases)

In [6]:
flatten = list(itertools.chain(*entities))
multi_word_entities = Counter(flatten).most_common()[0:200]

In [7]:
single_word_entities = []
for i in range(math.ceil(len(flatten)/10000)):
    single_word_entities.append(liteClient.getKeywords(' '.join(flatten[i*10000:(i+1)*10000])))

In [8]:
flatten2 = list(itertools.chain(*single_word_entities))
single_word_entities = Counter(flatten2).most_common()

In [9]:
max_ = 0
for item in multi_word_entities:
    if item[1]>max_:
        max_ = item[1]
multi_word_entities_normalized = []
for item in multi_word_entities:
    multi_word_entities_normalized.append((item[0],item[1]/max_))

In [10]:
max_ = 0
for item in single_word_entities:
    if item[1]>max_:
        max_ = item[1]
single_word_entities_normalized = []
for item in single_word_entities:
    single_word_entities_normalized.append((item[0],item[1]/max_))

In [11]:
all_entities = multi_word_entities_normalized+[entity for entity in single_word_entities_normalized if entity[0] not in flatten]

In [12]:
all_entities = [entity for entity in all_entities if entity[0] !='']

In [13]:
final_entities = np.zeros(1000)
dist = 0.9
while len(final_entities)>int(math.log2(len(df))):
    all_entities_sorted_by_weight = sorted(all_entities, key=lambda tup: tup[1], reverse=True)
    list1 = all_entities_sorted_by_weight[:]
    list2 = all_entities_sorted_by_weight[:]
    final_entities = []
    while list1:
        items1 = list1[0]
        x = None
        for items2 in list2:
            try:
                if items1 != items2:
                    if cosinus_distance(get_phrase_vec(items1[0]),get_phrase_vec(items2[0]))>dist:
                        if x:
                            x = (x[0]+[items2[0]], x[1]+items2[1])
                        else:
                            x = ([items1[0],items2[0]], items1[1]+items2[1])
                        list2.remove(items2)
                        list1.remove(items2)
            except:
                pass
        if x:
            final_entities.append(x)
        else:
            final_entities.append(([items1[0]],items1[1]))
        list1.remove(items1)
        list2.remove(items1)
    dist-=0.05
    if dist <= 0.4:
        break

In [14]:
sorted_final_entities = sorted(final_entities, key=lambda tup: tup[1], reverse=True)[:int(math.log2(len(df)))]

In [15]:
sorted_final_entities_headline = []
replacements = []
for tuple in sorted_final_entities:
    best_topic = ''
    best_topic_score = -1
    for item in tuple[0]:
        try:
            score = sum([d[word]for word in item.split()])*Counter(flatten)[item]
            if score>best_topic_score:
                best_topic=item
                best_topic_score=score
        except:
            pass
    replacements.append([best_topic,tuple[0]])
    sorted_final_entities_headline.append((best_topic,tuple[1]))

In [16]:
replacements

[['battery life',
  ['charge',
   'battery',
   'battery life',
   'charger',
   'battery charge',
   'charge hour']],
 ['step', ['step', 'k step', 'track step', 'step goal', 'measure']],
 ['sleep mode',
  ['sleep',
   'sleep mode',
   'sleep pattern',
   'sleep tracker',
   'sleep function',
   'activity sleep',
   'sleep habit',
   'sleep activity',
   'sleep log',
   'sleep quality',
   'sleep cycle']],
 ['band',
  ['band',
   'wrist band',
   'replacement band',
   'rubber band',
   'band clasp',
   'color band']],
 ['sleep track',
  ['track',
   'sleep track',
   'track sleep',
   'track activity',
   'way track',
   'track food']],
 ['customer service',
  ['customer service', 'customer', 'email', 'service', 'company', 'warranty']],
 ['time', ['time', 'sleep time', 'couple time']],
 ['flex', ['flex', 'flex flex', 'flex activity']],
 ['e g', ['e g', 'etc']],
 ['work', ['work', 'flex work']],
 ['activity',
  ['activity',
   'activity tracker',
   'activity level',
   'exercise',
   

In [17]:
for replacement in replacements:
    entities = [[replacement[0] if entity in replacement[1] else entity for entity in review_ents] for review_ents in entities]

In [18]:
df['topic_entities'] = entities
df['topic_entities_scores'] = entities_scores
df['topic_phrases'] = phrases
#df.to_pickle('./data/fitbitflexwirelessactivitysleepwristbandblack.pkl')

In [19]:
#filename = 'data/fitbitflexwirelessactivitysleepwristbandblack.json'

In [20]:
topics_filename = filename[:-5]+'_topics.pkl'

In [21]:
with open(topics_filename, 'wb') as f:
    pickle.dump([topic[0] for topic in sorted_final_entities_headline], f)

In [ ]:
#df.to_pickle("data/sonymdr7506professionallargediaphragmheadphone.pkl")

In [ ]:
#with open("data/sonymdr7506professionallargediaphragmheadphone_topics.pkl", 'wb') as f:
#    pickle.dump([topic[0] for topic in sorted_final_entities_headline], f)

In [ ]:
#flatten1 = list(itertools.chain(*topic_phrases))
#flatten2 = list(itertools.chain(*topic_entities_scores))
#d = {'np': flatten1, 'score': flatten2}
#lel = pd.DataFrame(data=d)
#lel = lel.drop_duplicates()
#lel.sort_values(by=['score'], ascending=False)